In [1]:
import numpy as np
import onnxruntime as ort
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.onnx
import torchvision.models as models
from onnxruntime.quantization import QuantType, quantize_dynamic

import onnx


### onnxruntime.quantization.preprocess

In [2]:
model = models.resnet18(pretrained=False)
model.eval()  # モデルを評価モードに設定
dummy_input = torch.randn(1, 3, 224, 224)
torch.onnx.export(model, dummy_input, "resnet18.onnx", verbose=True, input_names=['input'], output_names=['output'])

c:\Users\thyt\envs\onnx-cpu_torch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\thyt\envs\onnx-cpu_torch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [3]:
!python -m onnxruntime.quantization.preprocess --input resnet18.onnx --output resnet18-infer.onnx

In [4]:
model = onnx.load('resnet18.onnx')

num_layers = len(model.graph.node)
print(f"層の数: {num_layers}")

total_params = sum(np.prod(tensor.dims) for tensor in model.graph.initializer)
print(f"パラメータ数: {total_params}")

層の数: 65
パラメータ数: 11680872


In [5]:
model = onnx.load('resnet18-infer.onnx')

num_layers = len(model.graph.node)
print(f"層の数: {num_layers}")

total_params = sum(np.prod(tensor.dims) for tensor in model.graph.initializer)
print(f"パラメータ数: {total_params}")

層の数: 49
パラメータ数: 11680872


In [6]:
model_1 = onnx.load('resnet18.onnx')
model_2 = onnx.load('resnet18-infer.onnx')

# ノード（層）の名前とタイプを取得
model_1_nodes = {(node.name, node.op_type) for node in model_1.graph.node}
model_2_nodes = {(node.name, node.op_type) for node in model_2.graph.node}

diff_1_to_2 = model_1_nodes - model_2_nodes
diff_2_to_1 = model_2_nodes - model_1_nodes

print("モデル1にはあるがモデル2にはない層:")
for node in diff_1_to_2:
    print(f"block: {node[0]}, type: {node[1]}")

print("\nモデル2にはあるがモデル1にはない層:")
for node in diff_2_to_1:
    print(f"block: {node[0]}, type: {node[1]}")


モデル1にはあるがモデル2にはない層:
block: Identity_14, type: Identity
block: Identity_0, type: Identity
block: Identity_13, type: Identity
block: Identity_4, type: Identity
block: Identity_9, type: Identity
block: Identity_8, type: Identity
block: Identity_2, type: Identity
block: Identity_11, type: Identity
block: Identity_5, type: Identity
block: Identity_3, type: Identity
block: Identity_12, type: Identity
block: Identity_6, type: Identity
block: Identity_1, type: Identity
block: Identity_10, type: Identity
block: Identity_7, type: Identity
block: Identity_15, type: Identity

モデル2にはあるがモデル1にはない層:


In [7]:
import torch
import torchvision.models as models

# ResNet18モデルをロード
model = models.resnet18(pretrained=False)

# 層の数をカウント
num_layers = sum(1 for _ in model.modules())

# パラメータの総数をカウント
total_params = sum(p.numel() for p in model.parameters())

print(f"層の数: {num_layers}")
print(f"パラメータの総数: {total_params}")


層の数: 68
パラメータの総数: 11689512


In [8]:
import torchvision.models as models
from torchviz import make_dot

model = models.resnet18(pretrained=False)
dummy_input = torch.randn(1, 3, 224, 224)
output = model(dummy_input)
dot = make_dot(output, params=dict(list(model.named_parameters()) + [('input', dummy_input)]))
dot.render('resnet18_visualization', format='png')

'resnet18_visualization.png'